In [1]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup

# TASK 1

In [2]:
link = rq.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
BFS = BeautifulSoup(link, 'xml')
BFS_tab = BFS.find('table')
columns = ['Postalcode' ,'Borough' ,'Neighborhood']
df = pd.DataFrame(columns=columns)

In [4]:
for x in BFS_tab.find_all('tr'):
    list_yip=[]
    for k in x.find_all('td'):
        list_yip.append(k.text.strip())
    if len(list_yip)==3:
        df.loc[len(df)] = list_yip

In [5]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
df_new = df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df_new = df_new.reset_index(drop=False)
df_new.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [7]:
df = df.merge(df_new, how='left', on='Postalcode')

In [8]:
df.drop(['Neighborhood'],axis=1,inplace=True)

In [9]:
df = df.rename(columns={'Neighborhood_joined':'Neighborhood'})

In [10]:
df = df.rename(columns={'Neighborhood_joined':'Neighborhood'})

In [11]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [12]:
df.shape

(180, 3)

In [13]:
df = df[df.Borough != 'Not assigned']

In [14]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

In [16]:
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [ ]:
df.shape

# TASK 2

In [18]:
#reading the file to coord dataframe
df['Latitude'] = '0';
df['Longitude'] = '0';

cordinates = pd.read_csv('https://cocl.us/Geospatial_data')

In [19]:
for indx in df.index:
    for indxcord in cordinates.index:
        if df.iloc[indx, 0] == cordinates.iloc[indxcord, 0]:
            df.iloc[indx, 3] = cordinates.iloc[indxcord, 1]
            df.iloc[indx, 4] = cordinates.iloc[indxcord, 2]          

In [20]:
df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895


# TASK 3

In [21]:
df.Borough.value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [24]:
tab_analise = df.loc[df.Borough.isin(['Downtown Toronto', 'Central Toronto', 'West Toronto', 'East Toronto'])].copy()

In [25]:
tab_analise.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789
15,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
19,M4E,East Toronto,The Beaches,43.6764,-79.293


In [26]:
tab_analise.shape

(39, 5)

In [27]:
import folium

In [29]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(tab_analise['Latitude'],tab_analise['Longitude'],tab_analise['Borough'],tab_analise['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto